In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device("cuda")

In [2]:
import sys
sys.path.append('./utils/')
from extract_function_utils import *

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
# program_json_dict, program_id_lang_dic = read_program_tok_file()

In [ ]:
# function_json_dict, function_id_lang_dic = read_function_tok_file()

### Extract functions and create code_dicts

#### Generate Code_dicts

In [ ]:
program_json_dict, program_id_lang_dic = read_program_tok_file()
pids_dict, programs_dict, programs_toked_dict, programs_detoked_dict = get_all_programs_detok_from_cache(
                                                                                    program_id_lang_dic,
                                                                                    pids_dict_path, 
                                                                                    programs_dict_path, 
                                                                                    programs_toked_dict_path,
                                                                                    programs_detoked_dict_path)
# sys_calls_dict = get_sys_calls(sys_calls_path, programs_dict)
code_lang_dict = read_code_dict(programs_dict, program_id_lang_dic)

#### Tokenize Code_dicts

In [ ]:
tokenize_functions("code-dict", "functions", code_dict_path, langs=['Python'])

#### Save code_dicts to map_data

In [ ]:
code_lang_dict, code_id_lang_dic = read_toked_code_dict(code_dict_path)
# save_function_map_data(code_lang_dict, programs_dict, program_id_lang_dic)

#### Read code_dicts

In [ ]:
code_lang_dict, code_id_lang_dic = read_toked_code_dict(code_dict_path)

### Generate pairwise data

In [8]:
prepro_functions_dict = data_prepro_notok(functions_dict, functions_toked_dict)
exec_function_pids_dict, exec_prepro_functions_dict = get_exec_filtered_functions_dict(functions_pids_dict, 
                                                                                     prepro_functions_dict)

In [41]:
codex_lang_dict = {}
test_set = set(test_list)
for lang in langs:
    codex_lang_dict[lang] = {}
    pids = exec_function_pids_dict[lang]
    for i, pid in enumerate(pids):
        if pid in test_set:
            code_dic = code_id_lang_dic[lang][pid]
            function = "\n".join(code_dic['functions'])
            codex_lang_dict[lang][pid] = function

In [23]:
split_dict = load_split_dict()
test_list, val_list = get_eval_list(split_dict)

In [42]:
with open('codex_lang_dict_test.json', 'w') as outfile:
    json.dump(codex_lang_dict, outfile)

In [6]:
code_lang_dict, code_id_lang_dic = read_toked_code_dict(code_dict_path)
functions_pids_dict, functions_dict, functions_toked_dict, functions_detoked_dict = \
        get_all_functions_detok_from_cache(
            code_id_lang_dic, functions_pids_dict_path, functions_dict_path, 
            functions_toked_dict_path, functions_detoked_dict_path)

In [ ]:
split_dict = load_split_dict()
test_list, val_list = get_eval_list(split_dict)
prepro_functions_dict = data_prepro_notok(functions_dict, functions_toked_dict)
exec_function_pids_dict, exec_prepro_functions_dict = get_exec_filtered_functions_dict(functions_pids_dict, 
                                                                                       prepro_functions_dict)
get_pair_data_notok(data_path, "pair_data_notok_exec_function", 
                    exec_function_pids_dict, exec_prepro_functions_dict, test_list, val_list)

### Get sequence length (number of tokens)

In [ ]:
from transformers import AutoTokenizer
baseline_models = ['codebert', 'plbart', 'codet5']
tokenizer_dict = {model_type:AutoTokenizer.from_pretrained(baseline_model_dict[model_type])
                  for model_type in baseline_models}

In [ ]:
length_lang_dict = get_length_lang_dict(prepro_functions_dict, tokenizer_dict['codet5'], False) 

In [ ]:
# function_json_dict, function_id_lang_dic = read_function_tok_file()
# get_pairwise_functions(data_path, test_list, val_list, function_id_lang_dic)
# function_data = data_path + "pair_data_tok_function/"
# binarize(function_data, 
#          file_extensions.keys(), "code_prepro/bpe/cpp-java-python/vocab")

#### Get stats

In [ ]:
function_json_dict_filtered, _ = read_function_tok_file_filtered()
function_snippet_num_dict = {lang:[] for lang in langs}
function_line_num_dict = {lang:[] for lang in langs}
function_tokens_num_dict = {lang:[] for lang in langs}
function_desc_tokens_num_dict = {lang:[] for lang in langs}
function_num_dict = {}

for lang in langs:
    function_num_dict[lang] = len(function_json_dict_filtered[lang])
    for program in function_json_dict_filtered[lang]:
        function_str = "\n".join(program['code_dict']['functions'])
        lines = function_str.split('\n')
        function_line_num_dict[lang].append(len(lines))
        function_tokens_num_dict[lang].append(sum([len(x) for x in program['code_dict']['tokens']]))
print_stat("Average number of lines per function", function_line_num_dict)
print_stat("Average number of tokens per function",function_tokens_num_dict)
print_stat_dict("Number of functions", function_num_dict)


In [ ]:
# TODO
# run new baselines


In [ ]:
# test_set = set(test_list)
# val_set = set(val_list)
code_id_lang_dict = {}
for lang in langs:
    code_dicts = code_lang_dict[lang]
    code_id_dict = {}
    for code_dict in code_dicts:
        idx = code_dict["idx"].split("-")[0]
        code_id_dict[idx] = code_dict
    code_id_lang_dict[lang] = code_id_dict

In [ ]:
test_function_lang_dict = {}
for lang in langs:
    code_id_dict = code_id_lang_dict[lang]
    test_function_dict = {}
    for test_idx in test_list:
        if test_idx in code_id_dict:
            test_function_dict[test_idx] = "\n".join(code_id_dict[test_idx]["functions"])
    test_function_lang_dict[lang] = test_function_dict
            

In [ ]:
val_function_lang_dict = {}
for lang in langs:
    code_id_dict = code_id_lang_dict[lang]
    val_function_dict = {}
    for val_idx in val_list:
        if val_idx in code_id_dict:
            val_function_dict[val_idx] = "\n".join(code_id_dict[val_idx]["functions"])
    val_function_lang_dict[lang] = val_function_dict
            

In [ ]:
keys = list(val_function_lang_dict["Java"].keys())

In [ ]:
print(val_function_lang_dict["Java"][keys[0]])

In [ ]:
with open("val_functions.json", 'w') as outfile:
    json.dump(test_function_lang_dict, outfile)

In [ ]:
import matplotlib.pyplot as plt
def get_dist(input_list, n):
    count = 0
    l = []
    for i in range(1, n):
        count = 0
        for num in input_list:
            if num >= i:
                count += 1
        l.append((i, count/len(input_list)))
    return l

def show_plot(l):
    testList = l
    testList2 = [(elem1, elem2) for elem1, elem2 in testList]
    plt.scatter(*zip(*testList2))
    plt.show()